<a href="https://colab.research.google.com/github/monkrus/emotion_analyzer/blob/main/emotion_analyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**The following code analyzes the facial expression of the attached image. Run it and upload the image/s below the code.**

In [ ]:
!pip install google-generativeai pillow

# detect facial expression of the attached images
import google.generativeai as genai
from google.colab import userdata
from google.colab import files
from PIL import Image
import io

# Configure the API key
api_key = userdata.get('GEMINI_API_KEY')
if not api_key:
    print("Please add your API key to Colab's secrets manager with the name 'GOOGLE_API_KEY'")
    api_key = input("Or enter your API key here: ")
genai.configure(api_key=api_key)

# Initialize the model
model = genai.GenerativeModel('gemini-1.5-pro')

def analyze_facial_expression(image):
    # Prepare the prompt
    prompt = """
    Analyze this image and determine the facial expression of the person.
    Categorize it as one of the following: happy, sad, angry, surprised, neutral, or other.
    If 'other', specify what it might be.
    Also, provide a brief explanation of why you chose that category.
    Finally, estimate the confidence level of your analysis (low, medium, high).
    """

    # Generate content
    response = model.generate_content([prompt, image])

    # Print the response
    print(response.text)

# File upload widget
uploaded = files.upload()

# Process each uploaded file
for filename in uploaded.keys():
    print(f"Analyzing {filename}:")
    image = Image.open(io.BytesIO(uploaded[filename]))
    analyze_facial_expression(image)
    print("\n" + "="*50 + "\n")

# Optional: Analyze an image from a URL
def analyze_image_from_url(url):
    from urllib.request import urlopen
    from PIL import Image

    with urlopen(url) as response:
        image = Image.open(response)

    print(f"Analyzing image from URL: {url}")
    analyze_facial_expression(image)

# Uncomment and modify the line below to analyze an image from a URL
# analyze_image_from_url("https://example.com/path/to/image.jpg")

**The following code analyzes the facial expression of the attached video. Run it and upload the video/s below the code. (variant 1)**

In [ ]:
# Ensure you have the required libraries installed
!pip install google-generativeai pillow opencv-python-headless

import google.generativeai as genai
from google.colab import files
from google.colab import userdata
from PIL import Image
import io
import cv2
import tempfile
import time


# Configure the API key
api_key = userdata.get('GEMINI_API_KEY')
if not api_key:
    print("Please add your API key to Colab's secrets manager with the name 'GOOGLE_API_KEY'")
    api_key = input("Or enter your API key here: ")
genai.configure(api_key=api_key)

# Initialize the model
model = genai.GenerativeModel('gemini-1.5-pro')

def analyze_facial_expression(image):
    # Prepare the prompt
    prompt = """
    Analyze this image and determine the facial expression of the person.
    Categorize it as one of the following: happy, sad, angry, surprised, neutral, or other.
    If 'other', specify what it might be.
    Also, provide a brief explanation of why you chose that category.
    Finally, estimate the confidence level of your analysis (low, medium, high).
    """

    # Retry mechanism for handling rate limits
    retries = 5
    for i in range(retries):
        try:
            # Generate content
            response = model.generate_content([prompt, image])
            # Print the response
            print(response.text)
            return True
        except Exception as e:
            if "429" in str(e):  # Check for rate limit error
                delay = min(2**i, 30)  # Exponential backoff with a maximum delay of 30 seconds
                print(f"Rate limit exceeded, retrying in {delay} seconds...")
                time.sleep(delay)
            else:
                print(f"An error occurred: {e}")
                return False
    return False

# File upload widget
uploaded = files.upload()

# Process each uploaded video file
for filename in uploaded.keys():
    if filename.lower().endswith(('.mp4', '.avi', '.mov', '.mkv')):
        print(f"Analyzing {filename}:")

        # Save the uploaded video to a temporary file
        with tempfile.NamedTemporaryFile(delete=False, suffix=".mp4") as temp_video_file:
            temp_video_file.write(uploaded[filename])
            temp_video_file.flush()
            temp_video_path = temp_video_file.name

        video_capture = cv2.VideoCapture(temp_video_path)

        frame_count = 0
        frame_rate = int(video_capture.get(cv2.CAP_PROP_FPS))  # Get the frame rate of the video
        sample_rate = frame_rate * 5  # Analyze one frame every 5 seconds

        while video_capture.isOpened():
            ret, frame = video_capture.read()
            if not ret:
                break

            if frame_count % sample_rate == 0:
                # Convert the frame to PIL Image
                frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                image = Image.fromarray(frame_rgb)

                print(f"Analyzing frame {frame_count}:")
                if analyze_facial_expression(image):
                    print("\n" + "="*50 + "\n")
                else:
                    break  # Stop processing if an error occurs

            frame_count += 1

        video_capture.release()
    else:
        print(f"Unsupported file format: {filename}")


**The following code analyzes the facial expression of the attached video.Run it and upload the video/s below the code.(variant 2)**

In [ ]:
# Ensure you have the required libraries installed
!pip install google-generativeai pillow opencv-python-headless

import google.generativeai as genai
from google.colab import files
from google.colab import userdata
from PIL import Image
import io
import cv2
import tempfile
import time

# Configure the API key
api_key = userdata.get('GEMINI_API_KEY')
if not api_key:
    print("Please add your API key to Colab's secrets manager with the name 'GOOGLE_API_KEY'")
    api_key = input("Or enter your API key here: ")
genai.configure(api_key=api_key)

# Initialize the model
model = genai.GenerativeModel('gemini-1.5-pro')

def analyze_facial_expression(image):
    # Prepare the prompt
    prompt = """
    Analyze this image and determine the facial expression of the person.
    Categorize it as one of the following: happy, sad, angry, surprised, neutral, or other.
    If 'other', specify what it might be.
    Also, provide a brief explanation of why you chose that category.
    Finally, estimate the confidence level of your analysis (low, medium, high).
    """

    # Retry mechanism for handling rate limits and internal server errors
    retries = 5
    for i in range(retries):
        try:
            # Generate content
            response = model.generate_content([prompt, image])
            # Print the response
            print(response.text)
            return response.text
        except Exception as e:
            if "429" in str(e):  # Check for rate limit error
                delay = min(2**i, 30)  # Exponential backoff with a maximum delay of 30 seconds
                print(f"Rate limit exceeded, retrying in {delay} seconds...")
                time.sleep(delay)
            elif "500" in str(e):  # Check for internal server error
                delay = min(2**i, 30)  # Exponential backoff with a maximum delay of 30 seconds
                print(f"Internal server error, retrying in {delay} seconds...")
                time.sleep(delay)
            else:
                print(f"An error occurred: {e}")
                return None
    return None

# File upload widget
uploaded = files.upload()

# Process each uploaded video file
for filename in uploaded.keys():
    if filename.lower().endswith(('.mp4', '.avi', '.mov', '.mkv')):
        print(f"Analyzing {filename}:")

        # Save the uploaded video to a temporary file
        with tempfile.NamedTemporaryFile(delete=False, suffix=".mp4") as temp_video_file:
            temp_video_file.write(uploaded[filename])
            temp_video_file.flush()
            temp_video_path = temp_video_file.name

        video_capture = cv2.VideoCapture(temp_video_path)

        frame_count = 0
        frame_rate = int(video_capture.get(cv2.CAP_PROP_FPS))  # Get the frame rate of the video
        sample_rate = frame_rate * 5  # Analyze one frame every 5 seconds

        while video_capture.isOpened():
            ret, frame = video_capture.read()
            if not ret:
                break

            if frame_count % sample_rate == 0:
                # Convert the frame to PIL Image
                frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                image = Image.fromarray(frame_rgb)

                print(f"Analyzing frame {frame_count}:")
                if analyze_facial_expression(image):
                    print("\n" + "="*50 + "\n")
                else:
                    break  # Stop processing if an error occurs

            frame_count += 1

        video_capture.release()
    else:
        print(f"Unsupported file format: {filename}")
